# Python para el análisis de datos -  UNAV 2020-2021
---

# Notebook 6: Pandas, índices y métodos

## Índice  <a name="indice"></a>

- [Índices](#pandas_indices)
  - [Indexar con *.loc[]* y *iloc[]*](#indexar_loc_iloc)
  - [Renombrar etiquetas de índices y columnas](#renombrar_etiquetas)
- [Métodos avanzados](#pandas_metodos_avanzados)
  - [Métodos *.apply()* y *.map()*](#metodos_apply_map)
  - [Método *.copy()*](#metodo_copy)
  - [Método *.groupby()*](#metodo_groupby)
  - [Método *.agg()*](#metodo_agg)
  
- [Multi-índices](#pandas_multiindices)
  - [Método *.set_index()*](#pandas_multiindices_set_index)
  - [Método *.get_level_values()*](#pandas_multiindices_get_level_values)
  - [Método *.set_names()*](#pandas_multiindices_set_names)
  - [Indexar con *.loc()*](#pandas_multiindices_loc)
  
- [Ejercicios](#ejercicios)

## Índices<a name="pandas_indices"></a> 
[Volver al índice](#indice)

En la primera sección de esta sesión nos vamos a centrar en diferentes métodos para indexar _DataFrames_. Para mostrar estos métodos vamos a emplear el dataset de películas de James Bond, que contiene la siguiente información:

- Film: nombre de la película.
- Year: año.
- Actor: actor protagonista.
- Director: director de la película.
- Box Office: recaudación en millones de dólares.
- Budget: presupuesto en millones de dólares.
- Bond Actor Salary: salario del actor en millones de dólares.

In [ ]:
import pandas as pd
import numpy as np

df_bond = pd.read_csv("S6_datos/jamesbond.csv")
df_bond.head()

### Métodos *.set_index()* y *.reset_index()*

Por defecto el índice que se nos crea en un _DataFrame_ es numérico y va desde $0$ hasta $n-1$ filas. Podemos cambiar esto y poner nuestro propio índice utilizando el método *.set_index()*. El parámetro *inplace=True* realiza los cambios en el mismo _DataFrame_, y es equivalente a reasignar el resultado de *.set_index()* a la variable *df_bond*.

In [ ]:
df_bond.set_index("Film", inplace=True)
df_bond.head()

Podemos observar que nuestro índice ahora es la columna Film. Si queremos volver al estado anterior de nuestro _DataFrame_ podemos resetear el índice con el método *.reset_index()*.

In [ ]:
df_bond.reset_index(drop=False).head()

El parámetro *drop* le indica al método que el índice que se resetea, debe ser incluido de nuevo en el _DataFrame_. Por defecto *drop=False*.

In [ ]:
df_bond.sort_values('Budget', ascending=False).reset_index(drop=True).head()

In [ ]:
df_bond.reset_index(drop=False, inplace=False)
df_bond.head()

Podemos realizar el cambio de un índice a otro:

In [ ]:
df_bond.set_index("Film", inplace=True)
df_bond.head()

Es muy importante resetear el índice antes de volver realizar un *._set_index()* de otro índice.

In [ ]:
df_bond.reset_index(inplace=True)
df_bond.set_index("Year", inplace=True)
df_bond.head()

### Indexar con *.loc[]* y *iloc[]*<a name="indexar_loc_iloc"></a> 
[Volver al índice](#indice)

#### Indexar con *.loc[]* a través de etiqueta

Cargamos de nuevo el dataset indicando a pandas que utilice la columan "Film" como índice.

In [ ]:
df_bond = pd.read_csv("S6_datos/jamesbond.csv", index_col="Film")

df_bond.sort_index(inplace=True)
df_bond.head()

Podemos preguntar si un elemento está en el índice de forma análoga a como lo hacíamos con las listas:

In [ ]:
'Goldfinger' in df_bond.index

Podemos realizar un acceso a las filas del _DataFrame_ a través del índice, usando el indexador *_loc_*. Por ejemplo, esto nos va a permitir extraer información correspondiente a una película. El resultado de la extracción es un objeto _Series_, cuyos índices son los nombres de las columnas del _DataFrame_.

In [ ]:
s_bond_goldfinger = df_bond.loc["Goldfinger"]  #si quieres buscar algo por índice usas loc

print(type(s_bond_goldfinger))
s_bond_goldfinger

También podemos realizar el acceso a esos elementos como hacíamos en el apartado de _Series_:

In [ ]:
df_bond.loc["Goldfinger"]["Year"], df_bond.loc["Goldfinger"]["Budget"]

In [ ]:
df_bond.loc["Goldfinger"][["Year", "Budget"]]

Podemos extraer dos o más filas, incluso un rango de filas. En este caso, cuando el resultado es mayor de una fila, el objeto devuelto es un _DataFrame_ con la información a la que accedemos.

In [ ]:
df_bond_films = df_bond.loc[["Octopussy", "Moonraker", "Casino Royale"]]  #accediendo por índice

print(type(df_bond_films))
df_bond_films

In [ ]:
df_bond.loc[:"Dr. No"]

**¡Cuidado! Porque esto último funciona bien porque tenemos el _DataFrame_ ordenado alfabéticamente, si no lo estuviese el comportamiento puede ser inesperado.**

#### Indexar con *.iloc[]* a través de posición

In [ ]:
df_bond = pd.read_csv("S6_datos/jamesbond.csv")
df_bond.head()

De forma similar al acceso a través del índice, podemos acceder a un número de fila utilizando de *._iloc_[]*. De nuevo al acceder a una fila obtenemos un objeto _Series_.

In [ ]:
print(type(df_bond.iloc[0]))
df_bond.iloc[0]

In [ ]:
df_bond.iloc[-5:]  # coges las últimas 5 películas

El indexado funciona de forma análoga a una lista y podemos hacer _slicing_ y obtener rangos de datos.

In [ ]:
df_bond.iloc[10:15]

In [ ]:
df_bond.iloc[-5:]

¿Cuándo usar _loc_ o _iloc_? Las reglas básicas son:
* _.loc[]_ para indexado con etiqueta de índice.
* _.iloc[]_ para para indexado por posición de fila.

Estos métodos se pueden utilizar para acceder específicamente a una fila y una columna a la vez. Podemos imaginar que estamos accediendo a una celda de Excel (o un rango de celdas).

In [ ]:
df_bond.set_index('Film', inplace=True)
df_bond.head()

Para acceder al año y actor de una película:

In [ ]:
df_bond.loc['Dr. No', ['Year', 'Actor']]

In [ ]:
df_bond.iloc[0, :2]

Acceder a las películas 10 y 11, y las columnas (Director, Box Office, Budget):

In [ ]:
df_bond.iloc[10:12, 2:5]

También podemos modificar elementos de nuestro _DataFrame_ mediante el operador de asignación.

In [ ]:
df_bond.loc['A View to a Kill', 'Director'] = 'Juan Fernandez'
print(df_bond.loc['A View to a Kill'])  # te devuelve una serie 
df_bond[df_bond.index == 'A View to a Kill']  #te devuelve un dataset

In [ ]:
df_bond.iloc[1, 4] = 350
df_bond.head()

Se pueden realizar varios cambios a la vez:

In [ ]:
df_bond.loc["Dr. No", ["Box Office", "Budget", "Bond Actor Salary"]] = [448800000, 7000000, 600000]
df_bond.head()

Podemos realizar cambios a varias celdas a la vez utilizando el indexado booleano. Por ejemplo, vamos a actualizar el título nobiliario a Sean Connery:

In [ ]:
df_bond = pd.read_csv("S6_datos/jamesbond.csv")


mask = df_bond["Actor"] == "Sean Connery"
df_bond.loc[mask, "Actor"] = "Sir Sean Connery"

df_bond.loc[mask]

**¡Cuidado! Esto modifica el _DataFrame_, no devuelve una copia. Al indexar con _.loc[]_ no se devuelve un nuevo _Dataframe_, por el contrario, cuando realizamos _slicing_ sí devuelve una copia.**

In [ ]:
df_bond[df_bond['Actor'] == 'Sir Sean Connery']

### Renombrar etiquetas de índices y columnas<a name="renombrar_etiquetas"></a> 
[Volver al índice](#indice)

In [ ]:
df_bond = pd.read_csv("S6_datos/jamesbond.csv", index_col="Film")

df_bond.sort_index(inplace=True)
df_bond.head()

Podemos renombrar de forma sencilla una columna usando el metodo _.rename()_:

In [ ]:
df_bond.rename(columns={"Year" : "Release Date", "Box Office" : "Revenue"}, inplace=True)
df_bond.head()

In [ ]:
df_bond.rename(index={"Dr. No" : "Doctor No", 
                      "GoldenEye" : "Golden Eye",
                      "The World Is Not Enough" : "Best Bond Movie Ever"},
               inplace=True)

df_bond

El atributo *.columns* devuelve las columnas del _DataFrame_. Estas columnas pueden renombrarse:

In [ ]:
df_bond.columns

In [ ]:
df_bond.columns = ["Year of Release", "Director", "Gross Revenue", "Cost", "Actor", "Salary"]
df_bond.head()

## Métodos avanzados<a name="pandas_metodos_avanzados"></a> 
[Volver al índice](#indice)

### Métodos *.apply()* y *.map()*<a name="metodos_apply_map"></a> 
[Volver al índice](#indice)

In [ ]:
df_bond = pd.read_csv("S6_datos/jamesbond.csv", index_col="Film")

df_bond.sort_index(inplace=True)
df_bond.head()

Como vimos en los objetos _Series_, podemos utilizar el metodo _.map()_ para  mapear una función. Esto en _DataFrame_ podemos aprovecharlo y crear una nueva columna. Similar a la función *map()*, ésta nos permite utilizar funciones lambda o funciones definidas.

In [ ]:
df_bond['above_5M'] = df_bond['Bond Actor Salary'].map(lambda x: True if x > 5 else False)
df_bond.head()

Para _DataFrames_ existe un método llamado _.apply()_ que nos va a permitir actuar a nivel de fila. El concepto es similar a _.map()_ solo que ahora podemos acceder a todos los registros de la fila.

In [ ]:
def film_review(row):
    actor = row['Actor']
    budget = row['Budget']
    
    if actor == "Pierce Brosnan":
        return "Cool!"
    elif actor == "Roger Moore" and budget > 40:
        return "Okish"
    else:
        return "No idea"

In [ ]:
df_bond["film_review"] = df_bond.apply(film_review, axis="columns")

In [ ]:
df_bond.head()

Con este nuevo método hemos aumentado la versatilidad significativamente. La regla general sería:
- _.map()_ para aplicar una función en una columna.
- _.apply()_ para aplicar una función en una fila.


*.apply()* también se puede aplicar en columnas, aunque *.map()* es generalmente más rápido.

### Método *.copy()*<a name="metodo_copy"></a> 
[Volver al índice](#indice)

In [ ]:
df_bond = pd.read_csv("S6_datos/jamesbond.csv", index_col="Film")

df_bond.sort_index(inplace=True)
df_bond.head()

Como hemos visto en muchas ocasiones, para hacer un uso de memoria eficiente, Python no copia los valores del _DataFrame_, sino que produce una referencia al mismo. En ocasiones nos puede interesar realizar una copia del _DataFrame_, esto se hace con el método _.copy()_.

In [ ]:
df_bond_copied = df_bond.copy()

Ahora si modificamos algo, sólo se ve afectado uno de los _DataFrame_.

In [ ]:
df_bond.loc[df_bond['Actor'] == 'Roger Moore', 'Actor'] = 'Jeremy Irons'
df_bond.head()

In [ ]:
df_bond_copied.head()

### Método *.groupby()*<a name="metodo_groupby"></a> 
[Volver al índice](#indice)

Pandas radica en su sencillez a la hora de dividir, aplicar funciones y combinar de nuevo. Esto se conoce como la metodología _split-apply-combine_. Vamos a estudiar cómo aplicar todo esto con un _DataFrame_ de empresas americanas. El dataset es:

- Rank: posición de la compañía en la lista Fortune 1000.
- Company: nombre de la compañía.
- Sector: sector de la compañía.
- Industry: industria de la compañía.
- Location: ciudad donde se localizan los HQ.
- Revenue: ingresos en millones de dólares.
- Profits: beneficios en millones de dólares.
- Empleados: número de empleados.

In [ ]:
df_fortune = pd.read_csv("S6_datos/fortune1000.csv", index_col="Rank")

df_fortune.head()

In [ ]:
df_fortune.shape

Agrupamos por sectores y obtenemos un objeto *groupby*:

In [ ]:
sectors = df_fortune.groupby("Sector")

type(sectors)

Ahora de una forma muy sencilla se pueden aplicar funciones de cálculo a cada uno de los grupos.

In [ ]:
sectors.sum()

Se aplica la función _.sum()_ a cada uno de los grupos para todas las columnas numéricas, por eso obtenemos las tres de arriba.

In [ ]:
sectors["Revenue"].sum().head()

In [ ]:
sectors.describe()

Si queremos saber cuáles son los 5 que tienen la media mas alta, podemos simplemente ordenar el _DataFrame_ y sacar el resultado.

In [ ]:
sectors['Revenue'].sum().sort_values(ascending=False).head()

Las funciones _.head()_ y _.tail()_ son interesantes de aplicar, porque nos permiten acceder a los primeros/últimos $n$ registros de cada grupo. Si ordenamos el _DataFrame_ por ingresos y luego aplicamos una de estas funciones, después de agrupar podemos obtener, de forma sencilla, las empresas de cada sector que más ingresos tienen.

In [ ]:
sectors = df_fortune.sort_values('Revenue', ascending=False).groupby("Sector")
sectors.head(1).sort_values("Sector")

Podemos agrupar por más de un campo y realizar los cálculos de la misma forma.

In [ ]:
sectors = df_fortune.groupby(['Sector','Industry'])
sectors.median()

Si uno quiere transformar el objeto _groupby_ en un _DataFrame_, es tan sencillo como resetear el índice y ya tenemos un _DataFrame_ con nuestros cálculos.

In [ ]:
sectors.median().reset_index()

### Método *.agg()*<a name="metodo_agg"></a> 
[Volver al índice](#indice)

In [184]:
df_fortune = pd.read_csv("S6_datos/fortune1000.csv")
df_fortune = df_fortune.set_index("Rank", drop = True)
sectors = df_fortune.groupby("Rank")
df_fortune.head()

,Company,Sector,Industry,Location,Revenue,Profits,Employees
Rank,,,,,,,
1,Walmart,Retailing,General Merchandisers,"Bentonville, AR",482130,14694,2300000
2,Exxon Mobil,Energy,Petroleum Refining,"Irving, TX",246204,16150,75600
3,Apple,Technology,"Computers, Office Equipment","Cupertino, CA",233715,53394,110000
4,Berkshire Hathaway,Financials,Insurance: Property and Casualty (Stock),"Omaha, NE",210821,24083,331000
5,McKesson,Health Care,Wholesalers: Health Care,"San Francisco, CA",181241,1476,70400


El método *.agg()* nos va a permitir aplicar varios cálculos sobre diferentes columnas:

In [ ]:
sectors.agg({"Revenue" : ["sum", "mean"],
             "Profits" : "sum",
             "Employees" : "mean"})

Podemos aplicar _.apply()_ para los grupos. Si queremos calcular cuales son las empresas de cada sector que más profit generan. Usando _.groupby()_ podemos definir una función que llamaremos _ranker()_. Esta función etiqueta cada fila de $1$ a $n$, donde $n$ es el número de empresas en cada sector. Después, llamamos a _.apply()_ para aplicar la función a cada grupo (en este caso cada sector). 

In [ ]:
def ranker(df):
    """Asigna una posición en el ranking a cada empresa según 
    su profit siendo 1 la que mas profit genera.
    Asume que los datos estan ordenados de forma descendente."""
    
    df['sector_profit_rank'] = np.arange(1, len(df) + 1)
    return df

In [ ]:
df_fortune = pd.read_csv("S6_datos/fortune1000.csv")

df_fortune = df_fortune.sort_values('Profits', ascending=False)
df_fortune.head()

In [ ]:
df_fortune = df_fortune.groupby('Sector').apply(ranker)
df_fortune[df_fortune['sector_profit_rank'] == 1].head()

## Multi-índices<a name="pandas_multiindices"></a> 
[Volver al índice](#indice)

Los multi-índices nos permiten añadir más de un índice a nuestros _DataFrames_, esto nos sirve para categorizar los _DataFrames_ de mejor forma, ya sea, a través de más índices, o mediante más capas o layers.

Para ver el uso de multi-índices, cargamos un fichero que contiene el precio de la hamburguesa BigMac en varios países:

In [301]:
df_bigmac = pd.read_csv("S6_datos/bigmac.csv")

df_bigmac.sort_values('Price in US Dollars').head()

,Date,Country,Price in US Dollars
42,1/2016,Venezuela,0.66
98,7/2015,Venezuela,0.67
151,1/2015,Ukraine,1.20
139,1/2015,Russia,1.36
297,7/2013,India,1.50


In [302]:
df_bigmac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 652 non-null    object 
 1   Country              652 non-null    object 
 2   Price in US Dollars  652 non-null    float64
dtypes: float64(1), object(2)
memory usage: 15.4+ KB


Especificamos a Pandas que trate la columna "Date" como fecha, convertiéndola a tipo *datetime*:

In [303]:
df_bigmac = pd.read_csv("S6_datos/bigmac.csv", parse_dates=["Date"])
df_bigmac.head()

,Date,Country,Price in US Dollars
0,2016-01-01,Argentina,2.39
1,2016-01-01,Australia,3.74
2,2016-01-01,Brazil,3.35
3,2016-01-01,Britain,4.22
4,2016-01-01,Canada,4.14


In [ ]:
df_bigmac.info()

### Método *.set_index()*<a name="pandas_multiindices_set_index"></a> 
[Volver al índice](#indice)

Ya hemos utilizado *.set_index()* antes para crear un nuevo índice sobre un _DataFrame_, en este caso, lo vamos a hacer para crear uno múltiple, pero primero repasamos como funcionaba con un único parámetro:

In [304]:
df_bigmac_dates = df_bigmac.set_index(keys=["Date"])
df_bigmac_dates.head()


,Country,Price in US Dollars
Date,,
2016-01-01,Argentina,2.39
2016-01-01,Australia,3.74
2016-01-01,Brazil,3.35
2016-01-01,Britain,4.22
2016-01-01,Canada,4.14


In [305]:
df_bigmac_dates.loc['2016-01-01', 'Price in US Dollars'].mean()

3.303928571428571

Podemos ver que la columna "Date" se convierte en índice, porque se mueve a la izquierda y está en negrita. Vamos a crear un multiIndex con las columnas "Date" y "Country":

In [306]:
df_bigmac = pd.read_csv("S6_datos/bigmac.csv", parse_dates=["Date"])

df_bigmac.set_index(keys=["Date", "Country"], inplace=True)
df_bigmac.head()

Price in US Dollars
Date       Country                       
2016-01-01 Argentina                 2.39
           Australia                 3.74
           Brazil                    3.35
           Britain                   4.22
           Canada                    4.14

In [ ]:
df_bigmac.loc[('2016-01-01','Argentina')]

In [309]:
df_bigmac.sort_index(ascending=[True, False], inplace=True)
df_bigmac.head()

Price in US Dollars
Date       Country                           
2010-01-01 Uruguay                       3.32
           United States                 3.58
           Ukraine                       1.83
           UAE                           2.99
           Turkey                        3.83

Vamos a ver qué pasa si intentamos ordenar los índices. Usamos la función *.sort_index()*:

In [310]:
df_bigmac.sort_index(inplace=True)
df_bigmac.head()

Price in US Dollars
Date       Country                       
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76
           Britain                   3.67
           Canada                    3.97

Como vemos se ordenan de forma ascendente los dos valores, "Date" y "Country". Veamos como obtener información de los índices.

In [ ]:
df_bigmac.index[:5]

In [ ]:
df_bigmac.index.names

In [ ]:
type(df_bigmac.index)

### Método *.get_level_values()*<a name="pandas_multiindices_get_level_values"></a> 
[Volver al índice](#indice)

Este método nos permite obtener los valores para un índice o layer en concreto. Vamos a crear un multiIndex, pero usando una nueva forma, al leer el dataset con *.read_csv()* le pasamos directamente los índices que queremos usar.

In [311]:
df_bigmac = pd.read_csv("S6_datos/bigmac.csv", parse_dates=["Date"],
                        index_col=["Date", "Country"])

df_bigmac.sort_index(inplace=True)
df_bigmac.head()

Price in US Dollars
Date       Country                       
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76
           Britain                   3.67
           Canada                    3.97

In [312]:
df_bigmac.index.get_level_values("Date")

DatetimeIndex(['2010-01-01', '2010-01-01', '2010-01-01', '2010-01-01',
               '2010-01-01', '2010-01-01', '2010-01-01', '2010-01-01',
               '2010-01-01', '2010-01-01',
               ...
               '2016-01-01', '2016-01-01', '2016-01-01', '2016-01-01',
               '2016-01-01', '2016-01-01', '2016-01-01', '2016-01-01',
               '2016-01-01', '2016-01-01'],
              dtype='datetime64[ns]', name='Date', length=652, freq=None)

In [313]:
df_bigmac.index.get_level_values("Country")

Index(['Argentina', 'Australia', 'Brazil', 'Britain', 'Canada', 'Chile',
       'China', 'Colombia', 'Costa Rica', 'Czech Republic',
       ...
       'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'UAE', 'Ukraine',
       'United States', 'Uruguay', 'Venezuela', 'Vietnam'],
      dtype='object', name='Country', length=652)

### Método *.set_names()*<a name="pandas_multiindices_set_names"></a> 
[Volver al índice](#indice)

In [315]:
df_bigmac = pd.read_csv("S6_datos/bigmac.csv", parse_dates=["Date"],
                        index_col=["Date", "Country"])

df_bigmac.sort_index(inplace=True)
df_bigmac.head()

Price in US Dollars
Date       Country                       
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76
           Britain                   3.67
           Canada                    3.97

Este método nos permite cambiar el nombre de un índice ya creado en el _DataFrame_.

In [316]:
df_bigmac.index.set_names(["Fecha", "Pais"], inplace=True)
df_bigmac.head()

Price in US Dollars
Fecha      Pais                          
2010-01-01 Argentina                 1.84
           Australia                 3.98
           Brazil                    4.76
           Britain                   3.67
           Canada                    3.97

### Indexar con *.loc()*<a name="pandas_multiindices_loc"></a> 
[Volver al índice](#indice)

Vamos a ver cómo podemos obtener filas de un _DataFrame_ construido con un MultiIndex.

In [ ]:
df_bigmac = pd.read_csv("datos/bigmac.csv", parse_dates=["Date"],
                        index_col=["Date", "Country"])

df_bigmac.sort_index(inplace=True)
df_bigmac.head()

El método *.loc[]* puede acceder a una fila del _DataFrame_ y acepta etiquetas como índices; pero en nuestro caso, como tenemos dos índices, debemos usar una tupla para especificar las etiquetas para los índices que vamos a usar, y como último parámetro, especificamos el valor que queremos extraer (la serie o columna), de no ser así, nos devolvería todas las disponibles para las etiquetas suministradas.

In [ ]:
df_bigmac.loc[("2010-01-01", ["Brazil", "Argentina"]), "Price in US Dollars"]

In [ ]:
df_bigmac.loc[("2015-07-01", "Chile"), "Price in US Dollars"]

## Ejercicios<a name="ejercicios"></a> 
[Volver al índice](#indice)

#### Bloque 1: salarios en la ciudad de Chicago (chicago.csv)

Columnas:
- Name: nombre de la persona.
- Position Title: nombre del trabajo que realiza.
- Department: departamento en el que trabaja.
- Employee Annual Salary: salario de la persona.

1 - Calcula el salario medio de los habitantes de Chicago.

In [266]:
import pandas as pd
df_chicago = pd.read_csv("S6_datos/chicago.csv")
print(df_chicago.head())
df_chicago["Employee Annual Salary"] = df_chicago["Employee Annual Salary"].str.replace('$','').astype(float)
df_chicago["Employee Annual Salary"].mean()

                  Name            Position Title        Department  \
0      AARON,  ELVIA J          WATER RATE TAKER       WATER MGMNT   
1    AARON,  JEFFERY M            POLICE OFFICER            POLICE   
2       AARON,  KARINA            POLICE OFFICER            POLICE   
3  AARON,  KIMBERLEI R  CHIEF CONTRACT EXPEDITER  GENERAL SERVICES   
4  ABAD JR,  VICENTE M         CIVIL ENGINEER IV       WATER MGMNT   

  Employee Annual Salary  
0              $90744.00  
1              $84450.00  
2              $84450.00  
3              $89880.00  
4             $106836.00  


80204.178633899

2 - Calcula cuantos habitantes ganan más que la media. (_count()_ te puede ayudar)

In [ ]:
df_chicago[df_chicago["Employee Annual Salary"] > df_chicago["Employee Annual Salary"].mean()].count()
df_chicago[df_chicago["Employee Annual Salary"] > df_chicago["Employee Annual Salary"].mean()].shape[0]
np.count_nonzero(df_chicago["Employee Annual Salary"] > df_chicago["Employee Annual Salary"].mean())

3 - ¿Cuál es el departamento que emplea a mayor número de personas?

In [268]:
df_chicago.head()
df_chicago.groupby("Department").count().sort_values("Name",ascending = False).index[0]



,Name,Position Title,Department,Employee Annual Salary
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,90744.0
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,84450.0
2,"AARON, KARINA",POLICE OFFICER,POLICE,84450.0
3,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,89880.0
4,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,106836.0


In [ ]:
df_count = df_chicago.groupby("Department").count()
df_count[df_count == df_count.max]


np.argmax(df_count["Name"])
df_count.iloc[np.argmax(df_count["Name"])]

4 - ¿Cuál es el departamento que tiene un salario medio mayor?

In [ ]:
df_chicago.groupby("Department").mean().sort_values("Employee Annual Salary",ascending = False).index[0]
df_chicago.groupby("Department").mean().sort_values("Employee Annual Salary",ascending = False).head(1)


5 - Averigua cuales son los 5 departamentos con una media de salario mayor.

In [ ]:
df_chicago[["Employee Annual Salary","Department"]].groupby(
    "Department").mean().sort_values("Employee Annual Salary",ascending = False).head(5)
#df_chicago.groupby("Department").mean().sort_values("Employee Annual Salary",ascending = False).head(5)

6 - ¿Cuál es el departamento con más puestos de trabajo distintos?

In [ ]:
df_chicago.head(10)

df_chicago[["Department","Position Title"]].groupby(
    "Department").nunique().sort_values('Position Title', ascending = False).head(1)

7 - ¿Cuál es el trabajo mejor remunerado y en qué departamento se realiza?

In [ ]:

df_chicago[df_chicago["Employee Annual Salary"] == df_chicago["Employee Annual Salary"].max()][["Position Title", "Employee Annual Salary"]]
df_chicago.sort_values(["Employee Annual Salary"], ascending = False).iloc[0,1:]

8 - Averigua cuales son los trabajos de cada departamento que emplean a mayor número de personas.

In [ ]:
df_chicago.head(10)
df_chicago1 = df_chicago[["Department", "Position Title", "Name"]].groupby(["Department",
    "Position Title"]).count().sort_values('Name',ascending = False).groupby("Department").head(1)
df_chicago1

9 - Calcula el intervalo de confianza al 95% de los salarios de la ciudad de Chicago

In [ ]:
df_chicago.head()
std = df_chicago["Employee Annual Salary"].std()
mean = df_chicago["Employee Annual Salary"].mean()

from scipy import stats
f = stats.norm.ppf(0.975)

mean -f * std, mean + f * std


10 - ¿Cuál es el departamento con mayor variabiliad de salarios?

In [265]:
df_chicago[["Department","Employee Annual Salary"]].groupby(
    "Department").std().sort_values("Employee Annual Salary", ascending = False).head(1)

DataError: No numeric types to aggregate

11 - Sube un 10% el salario a aquellas personas que estén en el top 5 de salarios más bajos de su departamento.

In [373]:
df_chicago = pd.read_csv("S6_datos/chicago.csv")
# df_chicago = df_chicago.set_index("Department")
df_chicago["Employee Annual Salary"] = df_chicago["Employee Annual Salary"].str.replace('$', '').astype(float)
df_chicago.head()

,Name,Position Title,Department,Employee Annual Salary
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,90744.0
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,84450.0
2,"AARON, KARINA",POLICE OFFICER,POLICE,84450.0
3,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,89880.0
4,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,106836.0


In [394]:


df1 = df_chicago[["Department","Employee Annual Salary"]].sort_values([
    "Department","Employee Annual Salary"], ascending = True).set_index("Department", drop = False)
print(df1[:30])
df1.index.set_names("idx_department", inplace = True)
df2 = df1.groupby("Department")['Employee Annual Salary'].unique().to_dict()
df1.loc["AVIATION"]
for depart in df2.keys():
    df1.loc[(df1["Department"]==depart) &
            (df1['Employee Annual Salary'].isin(df2[depart][:5])), "Employee Annual Salary"] *= 1.1

print(df1[:30])

# df2.loc["AVIATION"]
# df2 = df2.set_index("Department")
# df2 = df2.groupby("Department")['Employee Annual Salary'].unique().to_dict()


    


                Department  Employee Annual Salary
Department                                        
ADMIN HEARNG  ADMIN HEARNG                 56544.0
ADMIN HEARNG  ADMIN HEARNG                 59184.0
ADMIN HEARNG  ADMIN HEARNG                 59184.0
ADMIN HEARNG  ADMIN HEARNG                 59184.0
ADMIN HEARNG  ADMIN HEARNG                 59184.0
ADMIN HEARNG  ADMIN HEARNG                 59184.0
ADMIN HEARNG  ADMIN HEARNG                 62004.0
ADMIN HEARNG  ADMIN HEARNG                 62004.0
ADMIN HEARNG  ADMIN HEARNG                 62004.0
ADMIN HEARNG  ADMIN HEARNG                 62004.0
ADMIN HEARNG  ADMIN HEARNG                 62004.0
ADMIN HEARNG  ADMIN HEARNG                 64992.0
ADMIN HEARNG  ADMIN HEARNG                 64992.0
ADMIN HEARNG  ADMIN HEARNG                 67212.0
ADMIN HEARNG  ADMIN HEARNG                 68028.0
ADMIN HEARNG  ADMIN HEARNG                 68028.0
ADMIN HEARNG  ADMIN HEARNG                 68028.0
ADMIN HEARNG  ADMIN HEARNG     

12 - Calcula la media del salario de los departamentos 'POLICE', 'POLICE BOARD' y 'FIRE'

In [403]:
df_chicago = pd.read_csv("S6_datos/chicago.csv", index_col = "Department")
# df_chicago = df_chicago.set_index("Department")
df_chicago["Employee Annual Salary"] = df_chicago["Employee Annual Salary"].str.replace('$', '').astype(float)
df_chicago.head()


df_chicago.loc["POLICE"].mean()
df_chicago.loc["POLICE BOARD"].mean()
df_chicago.loc["FIRE"].mean()

Employee Annual Salary    95700.627306
dtype: float64

#### Bloque 2: estudio tratamiento del cáncer (sanity.csv)

Columnas:
- idbus: identificador de la persona en el hospital. 
- sexo: 0 (hombre), 1 (mujer).
- edad: edad de la persona.
- altura: altura de la persona en cm.
- peso: peso de la persona en kg.
- tratamiento: 0 (quimioterapia), 1 (radioterapia).
- supervivencia: años de supervivencia desde el tratamiento.

In [405]:
df_sanity = pd.read_csv("S6_datos/sanity.csv")

14 - Calcula la supervivencia media de los participantes del estudio.

In [407]:
df_sanity.head()
df_sanity["supervivencia"].mean()

3.21

15 - Calcula cuantos sobreviven más que la media. ¿Y más que la mediana?. ¿Cuál es el porcentaje de cada uno de los casos?

In [426]:
df_sanity["supervivencia"].describe()
high_mean = df_sanity[df_sanity["supervivencia"] > df_sanity["supervivencia"].mean()].count()
high_median = df_sanity[df_sanity["supervivencia"] > df_sanity["supervivencia"].median()].count()
n_patients = df_sanity.count()
(high_mean/n_patients),(high_median/n_patients)

(idsub            0.355
 sexo             0.355
 edad             0.355
 altura           0.355
 peso             0.355
 tratamiento      0.355
 supervivencia    0.355
 dtype: float64,
 idsub            0.355
 sexo             0.355
 edad             0.355
 altura           0.355
 peso             0.355
 tratamiento      0.355
 supervivencia    0.355
 dtype: float64)

16 - ¿Cuál es el grupo de personas (hombre o mujer) con mayor supervivencia?

In [438]:
df_sanity.head()
df_sanity.groupby("sexo").count()  # sabiendo que la mitad son hombres y la mitad son mujeres
df_sanity.groupby("sexo").mean() # sexo 1 -> mujer. Las mujeres tienen una media de supervivencia superior

,idsub,edad,altura,peso,tratamiento,supervivencia
sexo,,,,,,
0.0,1024.5,43.48,174.09,73.27,0.54,2.94
1.0,1074.5,43.76,162.47,59.87,0.43,3.48


17 - ¿Qué tratamiento alarga más la supervivencia la quimioterapia o la radioterapia?

In [452]:
df_sanity.head()
# """Radio -> 1. Quimio -> 0"""

radio = df_sanity["supervivencia"][df_sanity["tratamiento"] == 1].mean()
quimio = df_sanity["supervivencia"][df_sanity["tratamiento"] == 0].mean()

radio, quimio

df_sanity.groupby("tratamiento")["supervivencia"].mean()  # Otra forma distinta de hacerlo


# radio, quimio

(tratamiento
 0.0    3.407767
 1.0    3.000000
 Name: supervivencia, dtype: float64,
 3.0,
 3.407766990291262)

18 - Asigna una nueva columna que marque como ancianos a los mayores de 60 años y jóvenes al resto.

In [453]:
df_sanity.head()

,idsub,sexo,edad,altura,peso,tratamiento,supervivencia
0,1000.0,0.0,61.0,166.0,74.0,1.0,2.0
1,1001.0,0.0,21.0,179.0,89.0,1.0,4.0
2,1002.0,0.0,59.0,166.0,59.0,1.0,2.0
3,1003.0,0.0,27.0,162.0,81.0,0.0,2.0
4,1004.0,0.0,23.0,164.0,69.0,1.0,2.0


In [455]:
df_sanity["anciano"] = df_sanity["edad"].map(lambda x: True if x > 60 else False)
df_sanity.head()

,idsub,sexo,edad,altura,peso,tratamiento,supervivencia,anciano
0,1000.0,0.0,61.0,166.0,74.0,1.0,2.0,True
1,1001.0,0.0,21.0,179.0,89.0,1.0,4.0,False
2,1002.0,0.0,59.0,166.0,59.0,1.0,2.0,False
3,1003.0,0.0,27.0,162.0,81.0,0.0,2.0,False
4,1004.0,0.0,23.0,164.0,69.0,1.0,2.0,False


19 - ¿Puedes ver si las mujeres que pesan más de 60 kilos sobreviven más que las que pesan menos?

In [462]:
print(df_sanity[df_sanity["peso"] > 60]["supervivencia"].mean())
print(df_sanity[df_sanity["peso"] < 60]["supervivencia"].mean())

3.162962962962963
3.293103448275862


20 - Crea una nueva columna que clasifique a las personas en base a su índice de masa corporal (imc) según el siguiente criterio:

- Por debajo de 18.5: por debajo del peso.
- 18.5 a 24.9: saludable.
- 25.0 a 29.9: Sobrepeso.
- 30.0 a 39.9: Obeso.
- Más de 40: Obesidad extrema o de alto riesgo.